In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [105]:
import math
from tqdm.notebook import tqdm
import sys
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd drive/MyDrive/datasets

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [3]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [4]:
text_list = word_tokenize(re.sub('[^a-zA-z0-9]', ' ', sent_tokenize(text)[0])); [text_list.index(i) for i in text_list]

[0, 1, 2, 3, 4, 5, 6, 7]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [5]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [6]:
onehot = torch.zeros([len(text_list), len(np.unique(word_tokenize(re.sub('[^a-zA-z0-9]', ' ', text))))]); onehot

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]])

In [7]:
uniword = np.unique(word_tokenize(re.sub('[^a-zA-z0-9]', ' ', text))).tolist()

In [8]:
for i in range(len(text_list)):
  if text_list[i] in uniword:
    indexx = uniword.index(text_list[i])
    onehot[i][indexx] = 1
print(onehot)

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])


1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [9]:
embedding = nn.Embedding(len(np.unique(word_tokenize(re.sub('[^a-zA-z0-9]', ' ', text)))), len(text_list))

In [10]:
for_indexes = torch.zeros(8)
for i in range(len(text_list)):
  if text_list[i] in uniword:
    for_indexes[i] = uniword.index(text_list[i])

In [11]:
for_indexes.long()

tensor([ 4, 24, 16,  6, 18, 22, 11,  8])

In [12]:
embedding(for_indexes.long())

tensor([[ 1.2179, -0.5288, -0.2822,  2.8217,  3.4018,  1.2108, -0.2459,  2.3452],
        [ 1.4329, -1.6946,  0.4885, -1.5158,  1.7172, -1.2646, -0.8125, -0.1755],
        [ 0.7794,  2.8786,  0.1736, -0.9336,  1.9461,  0.3923, -1.0480,  1.6091],
        [ 0.8953, -0.8241,  0.0416,  0.7857,  0.3699,  1.9490,  0.3454,  0.7366],
        [ 1.0175,  1.3943,  2.0876, -0.5446, -0.9292, -1.1559,  0.5718,  0.0385],
        [-0.1371, -1.6952, -0.4434,  0.1350,  0.7347, -0.1113,  0.7076, -0.1546],
        [ 0.3355,  1.5924, -1.2040,  1.3352,  0.5897, -0.2142,  1.0840, -0.2622],
        [ 0.8615, -0.4213, -0.3978, -0.4247,  1.6595, -1.6349, -0.4552,  0.0810]],
       grad_fn=<EmbeddingBackward0>)

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [13]:
data = pd.read_csv('surnames.csv')
data['nationality'] = pd.factorize(data['nationality'])[0]
result_df = data.drop_duplicates(subset=['surname'], keep='first')
y = result_df['nationality']
X = result_df['surname']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
class Vocab:
  def __init__(self, data):
        letters = ['<PAD>']
        for i in data:
            for j in i:
                if j not in letters:
                    letters.append(j)
        self.idx_to_token = {idx: val for idx, val in enumerate(letters)}
        self.token_to_idx = {val: idx for idx, val in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)
        self.max_seq_len = max([len(i) for i in data])

In [15]:
# vocab = Vocab(X)

In [16]:
class SurnamesDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab
    
    def vectorize(self, surname):
        abc = []
        for i in surname:
            if i in list(self.vocab.idx_to_token.values()):
                get_index = list(self.vocab.idx_to_token.values()).index(i)
                abc.append(get_index)
        while len(abc)<self.vocab.max_seq_len:
            abc.append(0)
        return torch.tensor(abc, dtype=torch.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.vectorize(self.X.iloc[index]), self.y.iloc[index]

In [17]:
# class Vocab:
#   def __init__(self, data: list):
#     unique_ch = ["<PAD>"] + list(set("".join(data)))
#     ch_ind = list(np.arange(len(unique_ch)))
#     self.idx_to_token = dict(zip(ch_ind, unique_ch))
#     self.token_to_idx = dict(zip(unique_ch, ch_ind))
#     self.vocab_len = len(unique_ch)
#     self.max_seq_len = max(list(map(len, data)))


# class SurnamesDataset(Dataset):
#   def __init__(self, X, y, vocab: Vocab):
#     self.X = X
#     self.y = y
#     self.vocab = vocab

#   def get_idx(self, surname):
#     idx = [0] * self.vocab.max_seq_len
#     for i, c in enumerate(surname):
#       idx[i] = self.vocab.token_to_idx[c]
    
#     return torch.tensor(idx, dtype=torch.int64)


#   def __len__(self):
#     return len(self.X)

#   def __getitem__(self, idx):
#     return self.get_idx(self.X.iloc[idx]), self.y.iloc[idx]

In [18]:
vocab = Vocab(X.tolist())
surname_train = SurnamesDataset(X_train, y_train, vocab)
surname_test = SurnamesDataset(X_test, y_test, vocab)


In [19]:
test_ds = SurnamesDataset(X_test, y_test, Vocab(X))
train_ds = SurnamesDataset(X_train, y_train, Vocab(X))

In [20]:
batch_size = 64
train_iter = DataLoader(train_ds, batch_size, shuffle=True)
test_iter = DataLoader(test_ds, batch_size, shuffle=False)

In [21]:
class fam_model(nn.Module):
    def __init__(self, input_size, vocab_size, num_channels, output_size):
        super(fam_model, self).__init__()
        self.input_size = input_size
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding = nn.Embedding(self.vocab_size, input_size)
        self.lr1 = nn.Conv1d(in_channels=input_size, out_channels=num_channels, kernel_size=3)
        self.lr2 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=3)
        self.lr3 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=3)
        self.lr4 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=3)
        self.relu = nn.ELU()
        self.norm = nn.BatchNorm1d(num_channels)
        self.maxpool = nn.MaxPool1d(kernel_size = 3)
        self.lr5 = nn.Linear(num_channels, self.output_size)
        
    def forward(self, x):
        out = self.embedding(x)
        out = self.lr1(out)
        out = self.norm(out)
        out = self.relu(out)
        out = self.lr2(out)
        out = self.norm(out)
        out = self.relu(out).squeeze(dim=2)
        out = self.lr5(out)
        return torch.nn.functional.log_softmax(out, dim=1)

In [60]:
## Модель 1

class sur_model(nn.Module):
    def __init__(self, input_size, vocab_size, num_channels, output_size):
        super(sur_model, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.seqnet = nn.Sequential(nn.Embedding(self.vocab_size, input_size),
            nn.Conv1d(input_size, 64, 3),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(64, 256, 2),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(256, 512, 2),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.Flatten(), 
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
            nn.Linear(128, 64)
        )
        self.fc = nn.Linear(64, output_size)


    def forward(self, x):
      out = self.seqnet(x)
      out = self.fc(out)
      return torch.nn.functional.log_softmax(out, dim=1)

In [61]:
input_size = vocab.max_seq_len
vocab_size = vocab.vocab_len
output_size = len(set(data['nationality']))
num_channels = 300
model = sur_model(input_size, vocab_size, num_channels, output_size)

In [62]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [63]:
def train(model, device, train_dl, optimizer, total):
    #set model in train() mode:
    model.train()
    
    total_loss = 0.0
    total_samples = 0.0
    correct_samples = 0.0
    
    for i, (inputs, targets) in tqdm(enumerate(train_dl), total= total, desc='Training minibatch loop '):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward and optimize        
        # zero grad before new step        
        optimizer.zero_grad()                        
        loss.backward()
        optimizer.step()   

        # calculating the total_loss for checking
        total_loss += loss           
        
        # PREDICTIONS 
        total_samples += targets.shape[0]   
        _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
        correct_samples += torch.sum(predictions_indices==targets)

    train_accuracy = float(correct_samples) / total_samples        
    
    return total_loss, train_accuracy                        

In [64]:
# TESTING THE MODEL
def test(model, device, test_dl, total):
    #set model in eval() mode (it skips Dropout etc):
    model.eval()
    
    total_samples = 0.0
    correct_samples = 0.0 
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for i, (inputs, targets) in tqdm(enumerate(test_dl), total=total, desc='Testing minibatch loop:'):                      
            # apply model to input data
            outputs = model(inputs)        
                       
            #PREDICTIONS
            total_samples += targets.shape[0]   
            _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
            correct_samples += torch.sum(predictions_indices==targets)                    
        
    test_accuracy = correct_samples / total_samples        
    
    return test_accuracy              

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [66]:
%%time
for_losses = []
n_total_steps = len(train_iter)


for epoch in range(25):
    total_loss, train_accuracy = train(model, device, train_iter, optimizer, total = math.ceil(len(train_ds)/64))
    test_accuracy = test(model, device, test_iter, total = math.ceil(len(test_ds)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [1/25], Loss: 176.7764, Train acc: 0.4902, Test acc: 0.5658


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

...

Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [11/25], Loss: 21.8774, Train acc: 0.9256, Test acc: 0.6314


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [87]:
class sur_model2(nn.Module):
    def __init__(self, input_size, vocab_size, num_channels, output_size):
        super(sur_model2, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.seqnet = nn.Sequential(nn.Embedding(self.vocab_size, input_size),
            nn.Conv1d(input_size, 64, 3),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(64, 256, 2),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(256, 512, 2),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.Flatten(), 
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
            nn.Linear(128, 64)
        )
        self.fc = nn.Linear(64, output_size)


    def forward(self, x):
      out = self.seqnet(x)
      out = self.fc(out)
      return torch.nn.functional.log_softmax(out, dim=1)

In [88]:
model = sur_model2(input_size, vocab_size, num_channels, output_size)

In [89]:
# подсказал саша волненко
layers = next(iter(model.children()))
for i, x in enumerate(layers):
    if i == 0:
        for param in x.parameters():
              param.requires_grad = False
        break

In [90]:
layers = next(iter(model.children()))

In [91]:
layers

Sequential(
  (0): Embedding(85, 17)
  (1): Conv1d(17, 64, kernel_size=(3,), stride=(1,))
  (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ELU(alpha=1.0)
  (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv1d(64, 256, kernel_size=(2,), stride=(1,))
  (6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ELU(alpha=1.0)
  (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Conv1d(256, 512, kernel_size=(2,), stride=(1,))
  (10): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ELU(alpha=1.0)
  (12): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (13): Flatten(start_dim=1, end_dim=-1)
  (14): Linear(in_features=512, out_features=256, bias=True)
  (15): ELU(alpha=1.0)
  (16): Linear(in_features=256, out_features=128, bias=True)
  (17): ELU(alpha=1.0)
  (18): Linear

In [92]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [93]:
%%time
for_losses = []
n_total_steps = len(train_iter)


for epoch in range(25):
    total_loss, train_accuracy = train(model, device, train_iter, optimizer, total = math.ceil(len(train_ds)/64))
    test_accuracy = test(model, device, test_iter, total = math.ceil(len(test_ds)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [1/25], Loss: 179.2348, Train acc: 0.4867, Test acc: 0.5271


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

...

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [24/25], Loss: 9.7966, Train acc: 0.9693, Test acc: 0.5934


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [25/25], Loss: 8.4994, Train acc: 0.9704, Test acc: 0.6093
Wall time: 56.2 s


## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [96]:
positive = pd.read_csv("negative_reviews.txt", sep="\n", header=None, names=["Review"])
nigative = pd.read_csv("positive_reviews.txt", sep="\n", header=None, names=["Review"])

In [98]:
positive["class"] = 1
nigative["class"] = 0
result = pd.concat([positive, nigative], ignore_index=True)

In [99]:
result

,Review,class
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
10657,a terrible movie that some people will neverth...,0
10658,there are many definitions of 'time waster' bu...,0
10659,"as it stands , crocodile hunter has the hurrie...",0
10660,the thing looks like a made-for-home-video qui...,0


In [101]:
X = result['Review']
y = result['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [142]:
words = []
for i in X:
    i = re.sub(r"[^a-zA-Z0-9\\]", " ", i)
    res = word_tokenize(i)
    self.max_seq_len = max(self.max_seq_len, len(res))
    words.append(res)
words = ['UNK']+[i[0] for i in words if i not in stopwords.words('english')]
uniqueWords = [] 
for i in words:
    if not i in uniqueWords:
        uniqueWords.append(i)
uniqueWords

['UNK',
 'the',
 'effective',
 'if',
 'emerges',
 'offers',
 'perhaps',
 'steers',
 'take',
 'this',
 'what',
 'wendigo',
 'one',
 'ultimately',
 'an',
 'illuminating',
 'a',
 'with',
 'not',
 'scores',
 'occasionally',
 'spiderman',
 'at',
 'it',
 'guaranteed',
 'light',
 'while',
 'though',
 'cantet',
 'ms',
 'like',
 'newton',
 'fuller',
 'compleja',
 'karmen',
 'gosling',
 'tender',
 'may',
 'in',
 'some',
 'sandra',
 'behind',
 'everytime',
 'manages',
 'singer',
 'you',
 'whether',
 'disney',
 'just',
 'morton',
 'part',
 'son',
 'haneke',
 'absorbing',
 'painful',
 'together',
 'director',
 'doug',
 'katz',
 'writer',
 'i',
 'chicago',
 'steve',
 'on',
 '4',
 'edited',
 'hip',
 'between',
 'somewhere',
 'digital',
 'unlike',
 'extreme',
 'run',
 'merely',
 'schweiger',
 'gloriously',
 'as',
 'far',
 'waydowntown',
 'tim',
 'more',
 'enormously',
 'here',
 'must',
 'ray',
 'everything',
 'sweet',
 'fans',
 'its',
 'stephen',
 'there',
 'remains',
 'makmalbaf',
 'for',
 'most',
 '

In [144]:
len(uniqueWords)

2175

In [159]:
class Vocab:
      def __init__(self, data):
                words = []
                self.max_seq_len = 0
                for i in data:
                    k = i
                    kk = nltk.word_tokenize(k)
                    self.max_seq_len = max(self.max_seq_len, len(kk))
                    i = re.sub(r"[^a-zA-Z0-9\\]", " ", i)
                    res = word_tokenize(i)
                    words.append(res)
                words = ['UNK']+[i[0] for i in words if i not in stopwords.words('english')]
                uniqueWords = [] 
                for i in words:
                    if not i in uniqueWords:
                        uniqueWords.append(i)
                self.idx_to_token = {idx: val for idx, val in enumerate(uniqueWords)}
                self.token_to_idx = {val: idx for idx, val in self.idx_to_token.items()}
                self.vocab_len = len(self.idx_to_token)

In [160]:
vocab = Vocab(X)

In [162]:
vocab.idx_to_token

{0: 'UNK',
 1: 'the',
 2: 'effective',
 3: 'if',
 4: 'emerges',
 5: 'offers',
 6: 'perhaps',
 7: 'steers',
 8: 'take',
 9: 'this',
 10: 'what',
 11: 'wendigo',
 12: 'one',
 13: 'ultimately',
 14: 'an',
 15: 'illuminating',
 16: 'a',
 17: 'with',
 18: 'not',
 19: 'scores',
 20: 'occasionally',
 21: 'spiderman',
 22: 'at',
 23: 'it',
 24: 'guaranteed',
 25: 'light',
 26: 'while',
 27: 'though',
 28: 'cantet',
 29: 'ms',
 30: 'like',
 31: 'newton',
 32: 'fuller',
 33: 'compleja',
 34: 'karmen',
 35: 'gosling',
 36: 'tender',
 37: 'may',
 38: 'in',
 39: 'some',
 40: 'sandra',
 41: 'behind',
 42: 'everytime',
 43: 'manages',
 44: 'singer',
 45: 'you',
 46: 'whether',
 47: 'disney',
 48: 'just',
 49: 'morton',
 50: 'part',
 51: 'son',
 52: 'haneke',
 53: 'absorbing',
 54: 'painful',
 55: 'together',
 56: 'director',
 57: 'doug',
 58: 'katz',
 59: 'writer',
 60: 'i',
 61: 'chicago',
 62: 'steve',
 63: 'on',
 64: '4',
 65: 'edited',
 66: 'hip',
 67: 'between',
 68: 'somewhere',
 69: 'digital',

In [164]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
        abc = []
        review = nltk.word_tokenize(review)
        for i in review:
            if i in list(self.vocab.idx_to_token.values()):
                get_index = list(self.vocab.idx_to_token.values()).index(i)
                abc.append(get_index)
        while len(abc)<self.vocab.max_seq_len:
            abc.append(0)
        return torch.tensor(abc, dtype=torch.int64)


  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.vectorize(self.X.iloc[idx]), self.y.iloc[idx]

In [165]:
vocab = Vocab(X_train.tolist())
RD_train = ReviewDataset(X_train, y_train, vocab)
RD_test = ReviewDataset(X_test, y_test, vocab)

In [220]:
RD_train[15]

(tensor([ 13, 155,  42, 269, 269, 870,   3, 365, 629,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0]),
 1)

In [221]:
batch_size = 32
train_dl = DataLoader(RD_train, batch_size, shuffle=True)
test_dl = DataLoader(RD_test, batch_size, shuffle=False)

In [222]:
i = next(iter(train_dl))
i[0][1]

tensor([   9,    3, 1729,  361,  141, 1497,    3,    3,  370,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0])

In [170]:
# подсказал Тимлфей Лашуков и Саша Волненко
class Reshaper(nn.Module):

  def __init__(self):
    super().__init__()

  def forward(self, x):
    x = torch.permute(x, (0, 2, 1))
    return x

In [253]:
class sur_model2(nn.Module):
    def __init__(self, input_size, vocab_size, output_size, emb_size):
        super(sur_model2, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.seqnet = nn.Sequential(nn.Embedding(self.vocab_size, emb_size),
            Reshaper(),                       
            nn.Conv1d(emb_size, 64, 3),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(64, 256, 2),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(256, 512, 2),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.Flatten(), 
            nn.Linear(3072, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
            nn.Linear(128, 64)
        )
        self.fc = nn.Linear(64, output_size)


    def forward(self, x):
      out = self.seqnet(x)
      out = self.fc(out)
      return out

In [254]:
input_size = vocab.max_seq_len
vocab_size = vocab.vocab_len
output_size = 2
emb_size = 64
model = sur_model2(input_size, vocab_size, output_size, emb_size)

In [255]:
criterion = nn.CrossEntropyLoss()   
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [259]:
def train(model, device, train_dl, optimizer, total):
    #set model in train() mode:
    model.train()
    
    total_loss = 0.0
    total_samples = 0.0
    correct_samples = 0.0
    
    for i, (inputs, targets) in tqdm(enumerate(train_dl), total= total, desc='Training minibatch loop '):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward and optimize        
        # zero grad before new step        
        optimizer.zero_grad()                        
        loss.backward()
        optimizer.step()   

        # calculating the total_loss for checking
        total_loss += loss           
        
        # PREDICTIONS 
        total_samples += targets.shape[0]   
        _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
        correct_samples += torch.sum(predictions_indices==targets)

    train_accuracy = float(correct_samples) / total_samples        
    
    return total_loss, train_accuracy                        

In [260]:
# TESTING THE MODEL
def test(model, device, test_dl, total):
    #set model in eval() mode (it skips Dropout etc):
    model.eval()
    
    total_samples = 0.0
    correct_samples = 0.0 
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for i, (inputs, targets) in tqdm(enumerate(test_dl), total=total, desc='Testing minibatch loop:'):                      
            # apply model to input data
            outputs = model(inputs)        
                       
            #PREDICTIONS
            total_samples += targets.shape[0]   
            _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
            correct_samples += torch.sum(predictions_indices==targets)                    
        
    test_accuracy = correct_samples / total_samples        
    
    return test_accuracy              

In [262]:
## %%time
for_losses = []
n_total_steps = len(train_iter)


for epoch in range(25):
    total_loss, train_accuracy = train(model, device, train_dl, optimizer, total = math.ceil(len(train_ds)/64))
    test_accuracy = test(model, device, test_dl, total = math.ceil(len(test_ds)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [1/25], Loss: 57.4314, Train acc: 0.9000, Test acc: 0.6005


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

...

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [24/25], Loss: 9.3008, Train acc: 0.9831, Test acc: 0.6458


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [25/25], Loss: 8.6631, Train acc: 0.9865, Test acc: 0.6333
